In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import silhouette_score, adjusted_rand_score, adjusted_mutual_info_score
import os

# Directorios de clustering
clustering_dirs = {
    "dbscan": "data/dbscan_clustering",
    "kmeans": "data/kmeans_clustering"
}

# Directorio de los features reducidos
features_dir = "data/reduction"

# Directorio de etiquetas reales
labels_dir = "etiquetas_reales"

# Cargar etiquetas reales desde CSV
def load_real_labels():
    label_files = {
        "train": os.path.join(labels_dir, "train_subset.csv"),
        "val": os.path.join(labels_dir, "val_subset.csv"),
    }
    
    real_labels = {}
    for key, path in label_files.items():
        if os.path.exists(path):
            df = pd.read_csv(path)
            if "label" in df.columns:
                real_labels[key] = dict(zip(df["youtube_id"], df["label"]))
            else:
                print(f"Advertencia: No se encontró la columna 'label' en {path}")
        else:
            print(f"Advertencia: Archivo {path} no encontrado.")

    return real_labels

# Cargar etiquetas reales
real_labels = load_real_labels()

# Evaluar cada método de clustering
for method, path in clustering_dirs.items():
    print(f"\nEvaluando {method.upper()}...\n")

    # Listar los archivos con etiquetas de clustering
    label_files = sorted([f for f in os.listdir(path) if f.endswith("_labels.npy")])

    for file in label_files:
        feature_file = file.replace(f"_{method}_labels.npy", ".npy")  # Obtener el archivo de features original
        feature_path = os.path.join(features_dir, feature_file)

        if not os.path.exists(feature_path):
            print(f"Archivo de features no encontrado para {file}.")
            continue

        # Cargar las etiquetas predichas y los features
        labels = np.load(os.path.join(path, file))
        features = np.load(feature_path)

        # Determinar si el archivo pertenece a train o val
        dataset_type = "train" if "train" in file else "val" if "val" in file else None

        # Obtener etiquetas reales correspondientes
        if dataset_type and dataset_type in real_labels:
            file_ids = [f.split("_")[0] for f in os.listdir(features_dir) if f.startswith(dataset_type)]
            ground_truth = np.array([real_labels[dataset_type].get(file_id, -1) for file_id in file_ids])

            # Verificar si hay etiquetas reales disponibles
            if len(set(ground_truth)) > 1 and len(set(labels)) > 1:
                silhouette = silhouette_score(features, labels)
                rand_index = adjusted_rand_score(ground_truth, labels)
                mutual_info = adjusted_mutual_info_score(ground_truth, labels)

                print(f"{file}:")
                print(f"  - Silhouette Score: {silhouette:.4f}")
                print(f"  - Rand Index: {rand_index:.4f}")
                print(f"  - Mutual Information: {mutual_info:.4f}\n")
            else:
                print(f"{file} no tiene etiquetas suficientes para calcular métricas de clustering.\n")
        else:
            print(f"{file} no tiene etiquetas reales disponibles, solo se calculará el Silhouette Score.")
            if len(set(labels)) > 1:
                silhouette = silhouette_score(features, labels)
                print(f"  - Silhouette Score: {silhouette:.4f}\n")
            else:
                print(f"  - {file} tiene un solo cluster, no se puede calcular Silhouette Score.\n")
